In [1]:
# create for run task 2.1 and task 2.2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip
import random
import time
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
f = 'drive/Shareddrives/EECS522/mnist.pkl.gz'

In [5]:

def load_data(f):
    
    '''
    load the MNIST data as a tuple contraining the training dta, evaluation data, and the test data.
    Training data has two entries, one is actual training images, which is a numpy ndarray with 50,000 entries, each entry is a numpy ndarray with 784 (28*28) pixels in a single MNIST image
                                  the other is numpy ndarray containing 50,000 entries. Those entries are just the digit values (0...9) for the corresponding images contained in the first entry of the tuple. 
    The ``validation_data`` and ``test_data`` are similar, except each contains only 10,000 images.
    
     This is a nice data format, but for use in neural networks it's helpful to modify the format of the ``training_data`` a little. That's done in the wrapper function ``load_data_wrapper()``, see
below.
    '''
    
    f = gzip.open(f, 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    
    return (training_data, validation_data, test_data)

def load_data_wrapper(f):
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data(f)
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y,10) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    
    return (training_data, validation_data, test_data)


def vectorized_result(j,n):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((n, 1))
    e[j] = 1.0
    return e

In [6]:
class Network(object):
    
    def __init__(self, sizes):
        
        
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        # initialize weights and biases
        # by different scenairos
        # yangsong gu 10/30/2021, 5:36PM

        if self.sizes[0] > 100:
            self.large_weight_initialization()
        else:
            self.small_weight_initialization()

        # for momentum based gradient descent
        self.weights_vel = [np.zeros((y, x))
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]
        self.biases_vel = [np.zeros((y, 1)) for y in self.sizes[1:]]
        
  
    def large_weight_initialization(self):

        # from nielson book, which is basic initilization approach, 
        # the weights is not standardized. 
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def small_weight_initialization(self):

        # standardized the weights, using sqrt(number of inputs)
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
            
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):

            a = self.sigmoid(np.dot(w, a)+b)

        return a
        
    def SGD(self, training_data, epochs, mini_batch_size, eta, subtype, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n_train = len(training_data)


        loss_Tr = [] # loss of training set  use upper case T to denote a set of samples
        loss_Te = [] # loss of test set
        acc_Tr  = [] # accuracy of training set
        acc_Te  = [] # accuracy of test set
    
        f = open('drive/Shareddrives/EECS522/newvision/{}_model_{}ep{}_mb{}_eta{}.txt'.format(subtype,self.sizes, epochs,mini_batch_size, eta), 'w')

        f.write('Layers {} learning rate: {}, min_batch: {}, epochs:{} \n'.format(self.sizes, 
                eta, mini_batch_size,epochs))
      

        f.write('epoch, loss_tr,loss_te, acc_tr, acc_te \n')
        
        if test_data:

            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n_train, mini_batch_size)]
            
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                
            if test_data:
                print("Epoch {}: {} / {}".format(j, self.evaluate(test_data), n_test))
                          
            else:
                print("Epoch {} complete".format(j))
            
            # compute the loss for training data set
            loss_tr = [0.5 * np.sum((self.feedforward(x) - y)**2) for (x, y) in training_data]

            loss_Tr.append(sum(loss_tr))
            
            
        
            # write the accuracy to 
            # compute the loss for test data set
            if test_data:
                loss_te = [0.5 * np.sum((self.feedforward(x) - vectorized_result(y,self.sizes[-1]))**2) for (x, y ) in test_data]
                loss_Te.append(sum(loss_te))
            # compute the accuracy for training data

            res_tr = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x,y) in training_data]
            # acc_tr = self.evaluate(training_data) / n_train
            acc_tr = sum(int(x == y) for (x, y) in res_tr) / n_train

            acc_Tr.append(acc_tr)
            
            # accuracy of test set
            if test_data:

                # res_te = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x,y) in test_data]
                # acc_te = sum(int(x == y) for (x, y) in res_te) / n_test

                acc_te = self.evaluate(test_data)/n_test
                acc_Te.append(acc_te)
            
            f.write("{}, {:10}, {:10}, {:.5}, {:.5}\n".format(j, sum(loss_tr), sum(loss_te),acc_tr, acc_te))
            
        f.close()
        return loss_Tr, loss_Te, acc_Tr, acc_Te
    
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def update_mini_batch_momentum(self, mini_batch,eta):

      # apply momentum to update weights and biases
      # ys g 6:20PM 10/30/2021
      nabla_b = [np.zeros(b.shape) for b in self.biases]
      nabla_w = [np.zeros(w.shape) for w in self.weights]
      for x, y in mini_batch:
          delta_nabla_b, delta_nabla_w = self.backprop(x, y)
          nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
          nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
      mu = 0.5
      self.weights_vel = [mu * v - (eta/len(mini_batch)) * nw for v, nw in zip(self.weights_vel, nabla_w)]
      self.biases_vel = [mu * v - (eta/len(mini_batch)) * nw for v, nw in zip(self.biases_vel, nabla_b)]

      self.weights  = [w  + v for w, v in zip(self.weights, self.weights_vel)]
      self.biases = [b + v for b, v in zip(self.biases, self.biases_vel)]

    
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            self.sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""

        # modeified by Yangsong 
        # y --> np.argmax(y) if there are multiple maximum
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        
        return sum(int(x == y) for (x, y) in test_results)
    
    
    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
    
    def sigmoid(self, z):
        '''simoid function'''
        return 1/(1+np.exp(-z))

    def sigmoid_prime(self, z):
        '''
        derivative of sigmoid function'''

        return self.sigmoid(z) * (1- self.sigmoid(z))

In [7]:
## TASK 2.2

In [ ]:
# adjust network structure 0
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_raw'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 30,30,30, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)


In [ ]:
# adjust network structure 1
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_raw'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 64,128,256, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

In [ ]:
# adjust network structure 2
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_raw'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 30, 30, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

In [12]:
# adjust network structure 3
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_raw'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 60,60,60, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 8523 / 10000
Epoch 1: 9299 / 10000
Epoch 2: 9408 / 10000
Epoch 3: 9510 / 10000
Epoch 4: 9569 / 10000
Epoch 5: 9586 / 10000
Epoch 6: 9467 / 10000
Epoch 7: 9609 / 10000
Epoch 8: 9620 / 10000
Epoch 9: 9631 / 10000
Epoch 10: 9659 / 10000
Epoch 11: 9544 / 10000
Epoch 12: 9657 / 10000
Epoch 13: 9681 / 10000
Epoch 14: 9665 / 10000
Epoch 15: 9617 / 10000
Epoch 16: 9663 / 10000
Epoch 17: 9656 / 10000
Epoch 18: 9686 / 10000
Epoch 19: 9664 / 10000
Epoch 20: 9621 / 10000
Epoch 21: 9659 / 10000
Epoch 22: 9689 / 10000
Epoch 23: 9681 / 10000
Epoch 24: 9667 / 10000
Epoch 25: 9679 / 10000
Epoch 26: 9674 / 10000
Epoch 27: 9676 / 10000
Epoch 28: 9712 / 10000
Epoch 29: 9693 / 10000
Epoch 30: 9685 / 10000
Epoch 31: 9690 / 10000
Epoch 32: 9688 / 10000
Epoch 33: 9668 / 10000
Epoch 34: 9689 / 10000
Epoch 35: 9677 / 10000
Epoch 36: 9694 / 10000
Epoch 37: 9665 / 10000
Epoch 38: 9702 / 10000
Epoch 39: 9707 / 10000
Epoch 40: 9700 / 10000
Epoch 41: 9703 / 10000
Epoch 42: 9698 / 10000
Epoch 43: 9693 / 1000

In [13]:
# adjust network structure 4
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_raw'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 60,60, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 9245 / 10000
Epoch 1: 9407 / 10000
Epoch 2: 9513 / 10000
Epoch 3: 9557 / 10000
Epoch 4: 9616 / 10000
Epoch 5: 9598 / 10000
Epoch 6: 9651 / 10000
Epoch 7: 9657 / 10000
Epoch 8: 9647 / 10000
Epoch 9: 9648 / 10000
Epoch 10: 9698 / 10000
Epoch 11: 9708 / 10000
Epoch 12: 9693 / 10000
Epoch 13: 9678 / 10000
Epoch 14: 9701 / 10000
Epoch 15: 9717 / 10000
Epoch 16: 9713 / 10000
Epoch 17: 9707 / 10000
Epoch 18: 9721 / 10000
Epoch 19: 9722 / 10000
Epoch 20: 9722 / 10000
Epoch 21: 9723 / 10000
Epoch 22: 9727 / 10000
Epoch 23: 9715 / 10000
Epoch 24: 9725 / 10000
Epoch 25: 9741 / 10000
Epoch 26: 9714 / 10000
Epoch 27: 9732 / 10000
Epoch 28: 9740 / 10000
Epoch 29: 9738 / 10000
Epoch 30: 9739 / 10000
Epoch 31: 9746 / 10000
Epoch 32: 9738 / 10000
Epoch 33: 9748 / 10000
Epoch 34: 9729 / 10000
Epoch 35: 9729 / 10000
Epoch 36: 9741 / 10000
Epoch 37: 9735 / 10000
Epoch 38: 9738 / 10000
Epoch 39: 9741 / 10000
Epoch 40: 9741 / 10000
Epoch 41: 9735 / 10000
Epoch 42: 9752 / 10000
Epoch 43: 9746 / 1000

In [14]:
# adjust network structure 5
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_raw'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)
print(time.time() - t1)

Epoch 0: 8487 / 10000
Epoch 1: 9178 / 10000
Epoch 2: 9392 / 10000
Epoch 3: 9501 / 10000
Epoch 4: 9578 / 10000
Epoch 5: 9608 / 10000
Epoch 6: 9642 / 10000
Epoch 7: 9644 / 10000
Epoch 8: 9688 / 10000
Epoch 9: 9702 / 10000
Epoch 10: 9643 / 10000
Epoch 11: 9729 / 10000
Epoch 12: 9725 / 10000
Epoch 13: 9742 / 10000
Epoch 14: 9750 / 10000
Epoch 15: 9774 / 10000
Epoch 16: 9740 / 10000
Epoch 17: 9734 / 10000
Epoch 18: 9773 / 10000
Epoch 19: 9769 / 10000
Epoch 20: 9743 / 10000
Epoch 21: 9779 / 10000
Epoch 22: 9754 / 10000
Epoch 23: 9761 / 10000
Epoch 24: 9770 / 10000
Epoch 25: 9785 / 10000
Epoch 26: 9789 / 10000
Epoch 27: 9775 / 10000
Epoch 28: 9792 / 10000
Epoch 29: 9789 / 10000
Epoch 30: 9793 / 10000
Epoch 31: 9784 / 10000
Epoch 32: 9780 / 10000
Epoch 33: 9780 / 10000
Epoch 34: 9782 / 10000
Epoch 35: 9791 / 10000
Epoch 36: 9796 / 10000
Epoch 37: 9783 / 10000
Epoch 38: 9788 / 10000
Epoch 39: 9795 / 10000
Epoch 40: 9770 / 10000
Epoch 41: 9781 / 10000
Epoch 42: 9781 / 10000
Epoch 43: 9786 / 1000

In [1]:
# tune learning rate

In [10]:
# adjust network structure 5
# min-batchsize = 20

model = {'min_batch':20, 'epoches':50, 'learningrate':0.1, 'comment': 'image_raw_learning_rate'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 1135 / 10000
Epoch 1: 1135 / 10000
Epoch 2: 1135 / 10000
Epoch 3: 1296 / 10000
Epoch 4: 3484 / 10000
Epoch 5: 2893 / 10000
Epoch 6: 3711 / 10000
Epoch 7: 5224 / 10000
Epoch 8: 6526 / 10000
Epoch 9: 7957 / 10000
Epoch 10: 8387 / 10000
Epoch 11: 8569 / 10000
Epoch 12: 8664 / 10000
Epoch 13: 8780 / 10000
Epoch 14: 8858 / 10000
Epoch 15: 8912 / 10000
Epoch 16: 8969 / 10000
Epoch 17: 9013 / 10000
Epoch 18: 9069 / 10000
Epoch 19: 9102 / 10000
Epoch 20: 9147 / 10000
Epoch 21: 9175 / 10000
Epoch 22: 9196 / 10000
Epoch 23: 9231 / 10000
Epoch 24: 9256 / 10000
Epoch 25: 9270 / 10000
Epoch 26: 9312 / 10000
Epoch 27: 9340 / 10000
Epoch 28: 9361 / 10000
Epoch 29: 9375 / 10000
Epoch 30: 9373 / 10000
Epoch 31: 9408 / 10000
Epoch 32: 9417 / 10000
Epoch 33: 9444 / 10000
Epoch 34: 9455 / 10000
Epoch 35: 9459 / 10000
Epoch 36: 9482 / 10000
Epoch 37: 9485 / 10000
Epoch 38: 9500 / 10000
Epoch 39: 9515 / 10000
Epoch 40: 9536 / 10000
Epoch 41: 9535 / 10000
Epoch 42: 9564 / 10000
Epoch 43: 9556 / 1000

In [14]:
# adjust network structure 5
# min-batchsize = 20
# >1000 small weights
model = {'min_batch':20, 'epoches':50, 'learningrate':3, 'comment': 'image_raw_learning_rate'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 8845 / 10000
Epoch 1: 9216 / 10000
Epoch 2: 9334 / 10000
Epoch 3: 9361 / 10000
Epoch 4: 9436 / 10000
Epoch 5: 9423 / 10000
Epoch 6: 9463 / 10000
Epoch 7: 9498 / 10000
Epoch 8: 9510 / 10000
Epoch 9: 9534 / 10000
Epoch 10: 9557 / 10000
Epoch 11: 9544 / 10000
Epoch 12: 9566 / 10000
Epoch 13: 9557 / 10000
Epoch 14: 9561 / 10000
Epoch 15: 9553 / 10000
Epoch 16: 9567 / 10000
Epoch 17: 9592 / 10000
Epoch 18: 9606 / 10000
Epoch 19: 9595 / 10000
Epoch 20: 9624 / 10000
Epoch 21: 9600 / 10000
Epoch 22: 9602 / 10000
Epoch 23: 9601 / 10000
Epoch 24: 9587 / 10000
Epoch 25: 9594 / 10000
Epoch 26: 9594 / 10000
Epoch 27: 9609 / 10000
Epoch 28: 9614 / 10000
Epoch 29: 9602 / 10000
Epoch 30: 9602 / 10000
Epoch 31: 9609 / 10000
Epoch 32: 9598 / 10000
Epoch 33: 9606 / 10000
Epoch 34: 9608 / 10000
Epoch 35: 9601 / 10000
Epoch 36: 9601 / 10000
Epoch 37: 9602 / 10000
Epoch 38: 9594 / 10000
Epoch 39: 9594 / 10000
Epoch 40: 9601 / 10000
Epoch 41: 9599 / 10000
Epoch 42: 9596 / 10000
Epoch 43: 9596 / 1000

In [16]:
# adjust network structure 5
# min-batchsize = 20

model = {'min_batch':20, 'epoches':50, 'learningrate':3, 'comment': 'image_raw_learning_rategr1000'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 9080 / 10000
Epoch 1: 9470 / 10000
Epoch 2: 9576 / 10000
Epoch 3: 9644 / 10000
Epoch 4: 9694 / 10000
Epoch 5: 9703 / 10000
Epoch 6: 9644 / 10000
Epoch 7: 9713 / 10000
Epoch 8: 9731 / 10000
Epoch 9: 9720 / 10000
Epoch 10: 9723 / 10000
Epoch 11: 9705 / 10000
Epoch 12: 9754 / 10000
Epoch 13: 9756 / 10000
Epoch 14: 9725 / 10000
Epoch 15: 9770 / 10000
Epoch 16: 9758 / 10000
Epoch 17: 9748 / 10000
Epoch 18: 9787 / 10000
Epoch 19: 9772 / 10000
Epoch 20: 9778 / 10000
Epoch 21: 9767 / 10000
Epoch 22: 9746 / 10000
Epoch 23: 9780 / 10000
Epoch 24: 9771 / 10000
Epoch 25: 9777 / 10000
Epoch 26: 9786 / 10000
Epoch 27: 9776 / 10000
Epoch 28: 9790 / 10000
Epoch 29: 9772 / 10000
Epoch 30: 9783 / 10000
Epoch 31: 9768 / 10000
Epoch 32: 9777 / 10000
Epoch 33: 9774 / 10000
Epoch 34: 9773 / 10000
Epoch 35: 9767 / 10000
Epoch 36: 9776 / 10000
Epoch 37: 9773 / 10000
Epoch 38: 9766 / 10000
Epoch 39: 9769 / 10000
Epoch 40: 9774 / 10000
Epoch 41: 9767 / 10000
Epoch 42: 9779 / 10000
Epoch 43: 9783 / 1000

In [7]:
# adjust network structure 5
# min-batchsize = 20

model = {'min_batch':20, 'epoches':50, 'learningrate':3, 'comment': 'image_raw_learning_ratele1000'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 8974 / 10000
Epoch 1: 9239 / 10000
Epoch 2: 9243 / 10000
Epoch 3: 9395 / 10000
Epoch 4: 9465 / 10000
Epoch 5: 9498 / 10000
Epoch 6: 9483 / 10000
Epoch 7: 9516 / 10000
Epoch 8: 9480 / 10000
Epoch 9: 9520 / 10000
Epoch 10: 9558 / 10000
Epoch 11: 9560 / 10000
Epoch 12: 9555 / 10000
Epoch 13: 9560 / 10000
Epoch 14: 9601 / 10000
Epoch 15: 9612 / 10000
Epoch 16: 9601 / 10000
Epoch 17: 9617 / 10000
Epoch 18: 9608 / 10000
Epoch 19: 9631 / 10000
Epoch 20: 9631 / 10000
Epoch 21: 9606 / 10000
Epoch 22: 9628 / 10000
Epoch 23: 9609 / 10000
Epoch 24: 9629 / 10000
Epoch 25: 9633 / 10000
Epoch 26: 9627 / 10000
Epoch 27: 9623 / 10000
Epoch 28: 9626 / 10000
Epoch 29: 9624 / 10000
Epoch 30: 9637 / 10000
Epoch 31: 9639 / 10000
Epoch 32: 9628 / 10000
Epoch 33: 9637 / 10000
Epoch 34: 9630 / 10000
Epoch 35: 9636 / 10000
Epoch 36: 9636 / 10000
Epoch 37: 9632 / 10000
Epoch 38: 9631 / 10000
Epoch 39: 9630 / 10000
Epoch 40: 9638 / 10000
Epoch 41: 9642 / 10000
Epoch 42: 9637 / 10000
Epoch 43: 9633 / 1000

In [10]:
#baseline

model = {'min_batch':10, 'epoches':100, 'learningrate':3, 'comment': 'image_baseline'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 30, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 8999 / 10000
Epoch 1: 9134 / 10000
Epoch 2: 9287 / 10000
Epoch 3: 9332 / 10000
Epoch 4: 9371 / 10000
Epoch 5: 9381 / 10000
Epoch 6: 9392 / 10000
Epoch 7: 9428 / 10000
Epoch 8: 9413 / 10000
Epoch 9: 9437 / 10000
Epoch 10: 9462 / 10000
Epoch 11: 9456 / 10000
Epoch 12: 9486 / 10000
Epoch 13: 9459 / 10000
Epoch 14: 9478 / 10000
Epoch 15: 9434 / 10000
Epoch 16: 9469 / 10000
Epoch 17: 9465 / 10000
Epoch 18: 9491 / 10000
Epoch 19: 9467 / 10000
Epoch 20: 9468 / 10000
Epoch 21: 9474 / 10000
Epoch 22: 9454 / 10000
Epoch 23: 9470 / 10000
Epoch 24: 9495 / 10000
Epoch 25: 9500 / 10000
Epoch 26: 9474 / 10000
Epoch 27: 9449 / 10000
Epoch 28: 9480 / 10000
Epoch 29: 9456 / 10000
Epoch 30: 9471 / 10000
Epoch 31: 9485 / 10000
Epoch 32: 9467 / 10000
Epoch 33: 9454 / 10000
Epoch 34: 9458 / 10000
Epoch 35: 9457 / 10000
Epoch 36: 9481 / 10000
Epoch 37: 9447 / 10000
Epoch 38: 9458 / 10000
Epoch 39: 9469 / 10000
Epoch 40: 9433 / 10000
Epoch 41: 9449 / 10000
Epoch 42: 9454 / 10000
Epoch 43: 9452 / 1000

In [ ]:
model = {'min_batch':20, 'epoches':50, 'learningrate':3, 'comment': 'image_momentum_0.5_again'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 6327 / 10000
Epoch 1: 6505 / 10000
Epoch 2: 6575 / 10000
Epoch 3: 6607 / 10000
Epoch 4: 6580 / 10000
Epoch 5: 6634 / 10000
Epoch 6: 6687 / 10000
Epoch 7: 6699 / 10000
Epoch 8: 6810 / 10000
Epoch 9: 7639 / 10000
Epoch 10: 8480 / 10000
Epoch 11: 8548 / 10000
Epoch 12: 8595 / 10000
Epoch 13: 9457 / 10000
Epoch 14: 9544 / 10000
Epoch 15: 9514 / 10000
Epoch 16: 9527 / 10000
Epoch 17: 9549 / 10000
Epoch 18: 9567 / 10000
Epoch 19: 9563 / 10000
Epoch 20: 9536 / 10000
Epoch 21: 9589 / 10000
Epoch 22: 9578 / 10000
Epoch 23: 9584 / 10000
Epoch 24: 9598 / 10000
Epoch 25: 9592 / 10000
Epoch 26: 9593 / 10000
Epoch 27: 9582 / 10000
Epoch 28: 9620 / 10000
Epoch 29: 9610 / 10000
Epoch 30: 9615 / 10000
Epoch 31: 9590 / 10000
Epoch 32: 9598 / 10000
Epoch 33: 9611 / 10000
Epoch 34: 9609 / 10000
Epoch 35: 9627 / 10000
Epoch 36: 9616 / 10000
Epoch 37: 9616 / 10000
Epoch 38: 9622 / 10000
Epoch 39: 9619 / 10000
Epoch 40: 9628 / 10000
Epoch 41: 9622 / 10000
Epoch 42: 9635 / 10000
Epoch 43: 9636 / 1000